# Classificação de Diabetes com o Dataset Pima Indians Diabetes

Neste notebook, utilizaremos o dataset Pima Indians Diabetes para construir um classificador de rede neural utilizando o `MLPClassifier` do scikit-learn. O fluxo do notebook inclui:
- Carregamento do dataset
- Preparação dos dados
- Pré-processamento (padronização)
- Treinamento do modelo
- Avaliação dos resultados

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

## Carregamento do Dataset

Utilizaremos o `fetch_openml` para carregar o dataset "PimaIndiansDiabetes". Esse dataset contém 768 amostras com 8 atributos numéricos e uma coluna alvo que indica se o paciente é diabético (1) ou não (0).

In [ ]:
# Carrega o dataset Pima Indians Diabetes
pima = fetch_openml(name='PimaIndiansDiabetes', version=1, as_frame=True)
df = pima.frame

# Exibe as primeiras linhas do dataset
print(df.head())

## Preparação dos Dados

Neste passo, separamos as features (atributos) da variável alvo e realizamos a divisão dos dados em conjuntos de treino e teste. Utilizaremos 80% dos dados para treino e 20% para teste, mantendo a proporção das classes com o parâmetro `stratify`.

In [ ]:
# Separa as features e a variável alvo
X = df.drop(columns='class')
y = df['class'].astype(int)  # Converte a coluna alvo para inteiro (0 ou 1)

# Divide os dados em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## Pré-processamento: Padronização

Como todas as colunas são numéricas, utilizamos o `StandardScaler` para padronizar os dados. Essa técnica transforma as features para terem média 0 e desvio padrão 1, o que pode ajudar na convergência do modelo.

In [ ]:
# Inicializa e ajusta o StandardScaler nos dados de treino
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Definição e Treinamento do Modelo

Utilizaremos o `MLPClassifier` do scikit-learn para construir uma rede neural com duas camadas ocultas. As camadas terão 64 e 32 neurônios, respectivamente. Configuramos a função de ativação `relu`, o solver `adam` e ativamos o early stopping para prevenir overfitting.

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(64, 32),
                      activation='relu',
                      solver='adam',
                      max_iter=200,
                      random_state=42,
                      early_stopping=True)

# Treina o modelo com os dados padronizados
model.fit(X_train_scaled, y_train)

## Avaliação do Modelo

Após o treinamento, avaliamos o desempenho do modelo utilizando o conjunto de teste. Calculamos a acurácia e geramos um relatório de classificação detalhado (precisão, recall, f1-score).

In [ ]:
# Realiza as previsões no conjunto de teste
y_pred = model.predict(X_test_scaled)

# Calcula a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")

# Exibe o relatório de classificação
print(classification_report(y_test, y_pred))